# CAD(2変数バージョン)

このスクリプトでは, CADを次の3つの段階に分けて行う.

- 射影段階: CADを与える多項式族の次数を下げる.
- 底段階:
- 持ち上げ段階:

まずは簡単のために多項式の**変数が2個の場合に限り**実装する.
その場合, 射影段階と持ち上げ段階の操作は1回で済む.

In [2]:
x,y = var("x y")
R = PolynomialRing(PolynomialRing(QQ,"y"),"x") # R=R[y][x]のこと.

# ここにCADしたい多項式のリストを渡す
#F = [x^2+y^2-1,x^2-y^3] 
F = [x^2-y,x*y-2]

## 射影段階

2変数多項式の族$ F $が与えられたとき, 1変数多項式の族$ F_1 \cup F_2 \cup F_3 $を次のように構成する.

- $ F_1 $: $ f \, (f \in F) $の変数yに対する係数の多項式全体
- $ F_2 $: $ \mathrm{gcd}(f, df/dx) \, (f \in F) $の変数yに対する係数の多項式全体
- $ F_3 $: $ \mathrm{gcd}(f, g) \, (f \neq g \in F) $の変数yに対する係数の多項式全体

In [3]:
# 変数xについてk次以下の部分を取り出す関数
def T(k,f):
    return f.mod(R(x^(k+1)))

# 与えられた関数をmonicにする

In [4]:
def projection(F:list):
    F = [R(f) for f in F]
    G = []
    
    # step1: F_1の計算
    for f in F:
        for term in f.coefficients():
            term = term/term.lc() # 先頭項を1にする(こういうのは2変数でしかダメっぽい)
            if term.degree()>0 and term not in G:
                G.append(term)
    # step2: F_2の計算 
    for f in F:
        for l in range(f.degree()+1):
            for k in range(l, f.degree()+1):
                for i in range(l+1):
                    if T(k,f).diff()==0:
                        continue
                    pscs = [term.leading_coefficient() for term in T(k,f).subresultants(T(k,f).diff())]
                    pscs = [term/term.lc() for term in pscs] # 先頭項を1にする
                    for psc in pscs: 
                        if psc.degree()>0 and psc not in G:
                            G.append(psc)
    # step3: F_3の計算
    for f in F:
        for g in F:
            if f==g:
                continue
            for l in range(min(f.degree(),g.degree())+1):
                for k in range(l,f.degree()+1):
                    for j in range(l, f.degree()+1):
                        if T(k,f)==0 or T(j,g)==0:
                            continue
                        pscs = [term.leading_coefficient() for term in T(k,f).subresultants(T(j,g))]
                        pscs = [term/term.lc() for term in pscs] # 先頭項を1にする
                        for psc in pscs:
                            if psc.degree()>0 and psc not in G:
                                G.append(psc)
    
    return G

[y, y^3 - 4]

In [123]:
#実験
G = projection(F);G

[y, y^3 - 4]

## 底段階

射影段階で求めた1変数多項式の族 $ G $ 不変な $ \mathbb{R} $ のセル分割を与える.
- 根の絶対値の限界を係数から評価する.
- "実根の分離"をすべて求める.
- 実根を解にもつ既約多項式を求める.


まず, スツルム列を用いた, 多項式 $ f(x) $ の $ (a,b] $ 上の実根の数を返す関数`count_root`を定義する.<p>
$ f(a) \neq 0, f(b) \neq 0 $ を満たしている必要があるので注意が必要である.

In [5]:
# 多項式f(x)の(a,b]上の異なる実根の数を返す関数
def count_root(f, a, b):
    if f in RR:
        return 0
    
    # スツルム列
    S = [f, f.diff()]
    
    while(True):
        if S[-2] % S[-1] == 0:
            break
        else:
            S.append(- (S[-2] % S[-1]))

    S_a = [s(a) for s in S]
    S_b = [s(b) for s in S]
    
    return v(S_a) - v(S_b)

# 実数列lstから符号の変化数v(lst)を計算する        
def v(lst):
    lst_sign = [bool(x>0) for x in lst if x!=0] #lstから符号列を計算
    count = 0
    for i in range(1,len(lst_sign)):
        if lst_sign[i] != lst_sign[i-1]:
            count += 1
    return count

多項式のからその解の絶対値の上界を計算する`bound_sol(f)`を定義する.

In [97]:
def bound_sol(f):
    f = f/f.lc()
    lst = f.coefficients(sparse=False)
    return 1 + max(abs(a/lst[-1]) for a in lst[:-1])

底段階の実装

In [125]:
def base(F:list, eps = 10^(-5)):
    # Fを既約多項式に分解して新たにIEE_Fとする:
    F = [f(y) for f in F]
    IRR_F = []
    for f in F:
        IRR_F += [g[0] for g in f.factor_list() if g[0] not in RR ]
    IRR_F = [PolynomialRing(QQ,"y")(f)for f in set(IRR_F)]
    
    # 解の絶対値の上界Mを求める
    M=max(bound_sol(f) for f in IRR_F)
    
    # 各既約多項式の解を求める.
    ans = []
    lst = [(-M,M,IRR_F)]
    while(len(lst)!=0):
        a,b,F = lst.pop()
        if(b-a<eps and len(F)==1):
            ans.append((a,b,F))
            continue
        if(len(F)==0):
            continue
        else:
            c = (a+b)/2
            F1 = [f for f in F if count_root(f,a,c)>0]
            F2 = [f for f in F if count_root(f,c,b)>0]
            if len(F1)>0:
                lst.append((a,c,F1))
            if len(F2)>0:
                lst.append((c,b,F2))
    return [(F[0],a,b) for a, b, F in ans] 
            

In [126]:
# 実験
base(G)

[(y^3 - 4, 832255/524288, 208065/131072), (y, -5/524288, 0)]

## 持ち上げ段階(未実装)

底段階で求めたセル分割を, $ \mathbb{R}^2 $ のセル分割に持ち上げる.



In [124]:
F

[x^2 - y, x*y - 2]

In [89]:
lst = [0,1,2]
lst.pop()

2